In [1]:
from datasets import load_dataset, load_from_disk

ModuleNotFoundError: No module named 'datasets'

In [12]:
mydataset = load_dataset("back2classroom/sidewalks")

Generating train split:   0%|          | 0/180000 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [13]:
mydataset.save_to_disk("images-tiff")

Saving the dataset (0/6 shards):   0%|          | 0/180000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [18]:
mydataset_reloaded = load_from_disk("images-tiff")
mydataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['filename', 'tif', 'tfw'],
        num_rows: 180000
    })
    val: Dataset({
        features: ['filename', 'tif', 'tfw'],
        num_rows: 20000
    })
})

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps
from patchify import patchify  #Only to handle large images
import random
from scipy import ndimage

In [4]:
# Load tiff stack images and masks
m = leafmap.Map(center=[40.7434679,-74.175889], zoom = 19) # NJIT
m.add_basemap("SATELLITE")

image = "images-tiff/satellite.tif"
m.user_roi_bounds()
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]
# tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

#165 large images as tiff image stack
large_images = tifffile.imread(image)
print(large_images.shape)


(1116, 1790, 3)


In [9]:
# Display the downloaded image on the map.
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

ImportError: localtileserver is not installed. Please install it before proceeding. https://github.com/banesullivan/localtileserver

In [ ]:
#Desired patch size for smaller images and step size.
patch_size = 256
step = 256

all_img_patches = []
for img in range(large_images.shape[0]):
    large_image = large_images[img]
    patches_img = patchify(large_image, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

    for i in range(patches_img.shape[0]):
        for j in range(patches_img.shape[1]):

            single_patch_img = patches_img[i,j,:,:]
            all_img_patches.append(single_patch_img)

images = np.array(all_img_patches)

#Let us do the same for masks
all_mask_patches = []
for img in range(large_masks.shape[0]):
    large_mask = large_masks[img]
    patches_mask = patchify(large_mask, (patch_size, patch_size), step=step)  #Step=256 for 256 patches means no overlap

    for i in range(patches_mask.shape[0]):
        for j in range(patches_mask.shape[1]):

            single_patch_mask = patches_mask[i,j,:,:]
            single_patch_mask = (single_patch_mask / 255.).astype(np.uint8)
            all_mask_patches.append(single_patch_mask)

masks = np.array(all_mask_patches)